In [4]:
import pandas as pd
import numpy as np

## Multilevel indexing

### Series

In [19]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data[['a', 'd']]

a  1    0.751802
   2   -1.817588
   3    0.448033
d  2   -1.057153
   3    0.115931
dtype: float64

In [11]:
data.loc[:,3]

a   -0.917635
b    0.409765
d   -0.026665
dtype: float64

In [12]:
data.unstack()

,1,2,3
a,0.353522,-0.195526,-0.917635
b,-0.038358,NaN,0.409765
c,-1.732114,0.290468,NaN
d,NaN,0.484624,-0.026665


### Data frames

In [20]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [24]:
frame.columns.names = ['state', 'color']
frame.index.names = ['k1', 'k2']

In [38]:
frame.swaplevel('k1', 'k2').sort_index()

state  Ohio     Colorado
color Green Red    Green
k2 k1                   
1  a      0   1        2
   b      6   7        8
2  a      3   4        5
   b      9  10       11

### Removing multilevel

In [49]:
frame.columns = frame.columns.droplevel(0)

In [53]:
frame.reindex()

color  Green  Red  Green
k1 k2                   
a  1       0    1      2
   2       3    4      5
b  1       6    7      8
   2       9   10     11

## Combining datasets

In [55]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [56]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [67]:
righth.index.names = ['key1', 'key2']
res = lefth.set_index(['key1', 'key2']).join(righth)
res

data  event1  event2
key1   key2                      
Nevada 2001   3.0     0.0     1.0
       2002   4.0     NaN     NaN
Ohio   2000   0.0     4.0     5.0
       2000   0.0     6.0     7.0
       2001   1.0     8.0     9.0
       2002   2.0    10.0    11.0

In [68]:
res['devent'] = res.event1.combine_first(res.data)
res

data  event1  event2  devent
key1   key2                              
Nevada 2001   3.0     0.0     1.0     0.0
       2002   4.0     NaN     NaN     4.0
Ohio   2000   0.0     4.0     5.0     4.0
       2000   0.0     6.0     7.0     6.0
       2001   1.0     8.0     9.0     8.0
       2002   2.0    10.0    11.0    10.0